## Replicando experimentos en Seldon* paper

In [1]:
import sys
sys.path.append("/home/pablo/tesis/tsm-pipeline/code")
%load_ext autoreload
%autoreload 2

import docker
docker_client = docker.from_env()

In [2]:
# Configura todo lo necesario en el ambiente
from scripts.xxx import run_tsm, ExperimentSettings
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
    "o11y_db_dir": "/home/pablo/tesis/tsm-pipeline/code/tsm.db",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

## Generando sets de entrenamiento

Usando las librerias de scikit learn para partir el dataset en train y test, usando `from sklearn.model_selection import train_test_split`. Aprovechando que `/home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100.txt` ya esta todo descargado, voy a usar este dataset como el total.

In [4]:
!cat /home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100.txt|head -n10

1046224544/fontend/eb7f2c0e136477ebf8ebcfe9050bd2fb586fe46b
1999/node-couchdb/4825580ec2d8cd8b1c490266f4ced3395c2cc8ee
3dbeb41841bfbfcc24d55143816cf7f1/atl-wdi-10/2f0c30bc082c0d1c3508cb7528459efc97633c65
4minitz/4minitz/cc1dfc424fa5df3719df3a1e0f43eb039996bce6
5techcenter/reactcms/1e163bc84fa7e874369da4c8e0077131484274e3
abansal98/datapool/5d0aeb7b735858a82ec3f4aca954827b21d5e5a2
abdizriel/nodejs-microservice-starter/3fa89d901930f68e7708f4b29e563a858fdaf316
acm309/putongoj/920a7a5885d59a3a31fec08380eaeff3189146c9
adnanrahic/a-crash-course-on-serverless-auth/83b16eba895a1d05cca588b4dc2487db7ce2c5b4
adnanrahic/building-a-serverless-rest-api-with-nodejs/8fcd380e090c07729f3c401103d2f53dbbfccdd2


In [3]:
def read_dbs_dataset(path):
    dbs = []
    with open(path, "r") as f:
        for l in f.readlines():
            dbs.append(l.rstrip())
    return dbs

nosql_top_100 = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100.txt")
len(nosql_top_100)

100

El codigo debajo parte el data set en 50% para entranamiento, 50% para test.

In [4]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

train, test = train_test_split(nosql_top_100, train_size=.5, shuffle=True, random_state=rand)

In [5]:
results_dir = '/home/pablo/results/seldonstar_1_re'

In [6]:
# training
train_settings = ExperimentSettings(
    name="seldonstar_1_re",
    query_type="nosql",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

2023-04-27 00:23:56,756 - INFO - creating /home/pablo/results/seldonstar_1_re dir. Omitting error if already existing
2023-04-27 00:23:56,758 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-04-27 00:23:57,232 - INFO - running at container 9e8832292f422efb3941acb1ab0563aed8892587a190cfc38209d1575fc124ad. Use `docker logs 9e8832292f422efb3941acb1ab0563aed8892587a190cfc38209d1575fc124ad --tail 10 --follow` to follow progress
2023-04-27 00:42:31,150 - INFO - running combine scores
g: /home/pablo/results/seldonstar_1_re/*/NosqlInjectionWorse-*/reprScores.txt


In [44]:
!cat /home/pablo/results/seldonstar_1/averaged-results.csv|wc -l

38


In [8]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql",
    cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.10.5",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

Evaluating worse with 50 dbs
2023-04-25 23:48:18,533 - INFO - creating /home/pablo/results/seldonstar_1/worse dir. Omitting error if already existing
2023-04-25 23:48:19,326 - INFO - Starting processing with 4 processes
2023-04-25 23:48:19,356 - INFO - Evaluating ailoitte - node-mongodb-graphql-starter
2023-04-25 23:48:19,356 - INFO - Evaluating callicoder - node-easy-notes-app
2023-04-25 23:48:19,356 - INFO - Evaluating brickyang - egg-mongo-native
2023-04-25 23:48:19,356 - INFO - Evaluating buunguyen - mongoose-deep-populate
2023-04-25 23:48:42,783 - INFO - Evaluating attriumph - fullstack-tiny-url-service
2023-04-25 23:48:43,486 - INFO - Evaluating apilayer - numvalidate
2023-04-25 23:48:43,838 - INFO - Evaluating aravindnc - mongoose-paginate-v2
2023-04-25 23:48:44,453 - INFO - Evaluating alexzeitler - mongoose-schema-reference-sample
2023-04-25 23:49:05,743 - INFO - Evaluating blinktaginc - node-gtfs
2023-04-25 23:49:06,352 - INFO - Evaluating adnanrahic - nodejs-restful-api
2023-

In [9]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

2023-04-26 00:01:10,662 - INFO - Result sets sizes: Worse 542, Boosted 79, V0 480, All 978
2023-04-26 00:01:10,685 - INFO - Score results: Precision: 0.0000. Recall: 0.0000. Accuracy: 0.4284


(0.0, 0.0, 0.4284253578732106)

## Prueba dos con otro set de entrenamiento, del mismo tamaño

In [10]:
train2, test2 = train_test_split(nosql_top_100, train_size=.5, shuffle=True, random_state=rand)

In [13]:
def run_nosql_training_eval_pipeline(name, train, test, results_dir):
    # training
    train_settings = ExperimentSettings(
        name=name,
        query_type="nosql",
        project_list=train,
        results_dir=results_dir,
        **defaults
    )

    run_tsm(docker_client, train_settings, block=True)

    # evaluation
    external_predicate_file = path_join(results_dir, 'averaged-results.csv')
    worse_settings = EvaluationSettings(
        search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql",
        cli_version="2.5.2",
        query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
        external_predicate_file=external_predicate_file,
        **defaults_evalute,
    )

    print("Evaluating worse with %d dbs" % (len(test)))
    evaluate(
        settings=worse_settings,
        output_dir=path_join(results_dir, 'worse'),
        dbs=test,
    )

    v0_settings = EvaluationSettings(
        search_path="/home/pablo/codeqlv0/javascript/ql/lib",
        cli_version="2.10.5",
        query_file="/home/pablo/tesis/tsm-pip0eline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
        **defaults_evalute,
    )

    print("Evaluating v0 with %d dbs" % (len(test)))
    evaluate(
        settings=v0_settings,
        output_dir=path_join(results_dir, 'v0'),
        dbs=test,
    )

In [14]:
calculate_scoressql_training_eval_pipeline("seldonstar_2", train2, test2, '/home/pablo/results/seldonstar_2')

2023-04-26 09:47:59,715 - INFO - creating /home/pablo/results/seldonstar_2 dir. Omitting error if already existing
2023-04-26 09:47:59,717 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-04-26 09:48:00,254 - INFO - running at container d53bd3257522f284101c36612b6c7f8b9d84b78cf54bb8c18726b843a53c6ad6. Use `docker logs d53bd3257522f284101c36612b6c7f8b9d84b78cf54bb8c18726b843a53c6ad6 --tail 10 --follow` to follow progress
2023-04-26 10:07:34,728 - INFO - running combine scores
g: /home/pablo/results/seldonstar_2/*/NosqlInjectionWorse-*/reprScores.txt
Evaluating worse with 50 dbs
2023-04-26 10:07:34,789 - INFO - creating /home/pablo/results/seldonstar_2/worse dir. Omitting error if already existing
2023-04-26 10:07:35,598 - INFO - Starting processing with 4 processes
2023-04-26 10:07:35,628 - INFO - Evaluating amazingandyyy - mern
2023-04-26 10:07:35,628 - INFO - Evaluating andriichyzh - node-js-advanced-training
2023-04-26 10:07:35,628 - INFO - Evaluating adnanrahi

In [15]:
calculate_scores(
    results_folder='/home/pablo/results/seldonstar_2',
    cleanup_base_dir="/tmp",
)

2023-04-26 10:26:09,918 - INFO - Result sets sizes: Worse 155, Boosted 58, V0 132, All 258
2023-04-26 10:26:09,924 - INFO - Score results: Precision: 0.0000. Recall: 0.0000. Accuracy: 0.2636


(0.0, 0.0, 0.26356589147286824)

Los resultados dados por `train2` y `test2` son los siguientes:
```
2023-04-26 10:26:09,918 - INFO - Result sets sizes: Worse 155, Boosted 58, V0 132, All 258
2023-04-26 10:26:09,924 - INFO - Score results: Precision: 0.0000. Recall: 0.0000. Accuracy: 0.2636
```

Por lo visto hay muchas DBs que un modeo vacío como resultado, lo cual lleva a que sean inutilizables. Por ejemplo, viendo la detalles acerca de los resultados con http://localhost:8888/lab/tree/notebooks/dev/debugging_zero_scores.ipynb, 25 dbs de test dieron cero alarmas en worse.